In [38]:
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os
import random
import face_recognition
import time

In [39]:
DETECT_CONF_THRESH = 0.5
RECOG_CONF_THRESH = 0.5
image_folder = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/datasets/caltech_faces_test/"
detector_main_folder = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/DL_dlib_SVM/face_detection_model/"
encodings_save_path = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/DL_dlib_SVM/output/caltech_train_encodings.pickle"
recognizer_save_path = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/DL_dlib_SVM/output/caltech_train_recognizer.pickle"
label_encoder_save_path = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/DL_dlib_SVM/output/caltech_train_label_encoder.pickle"


In [40]:
# Load face detector model
print("[INFO] loading face detector...")
protoPath = os.path.join(detector_main_folder, "deploy.prototxt")
modelPath = os.path.join(detector_main_folder,"res10_300x300_ssd_iter_140000.caffemodel")
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
print("[INFO] face detector loaded.")
# load the actual face recognition model along with the label encoder
print("[INFO] loading trained face recognizer model...")
recognizer = pickle.loads(open(recognizer_save_path, "rb").read())
le = pickle.loads(open(label_encoder_save_path, "rb").read())



[INFO] loading face detector...
[INFO] face detector loaded.
[INFO] loading trained face recognizer model...


In [43]:
# load the image, resize it to have a width of 600 pixels (while
# maintaining the aspect ratio), and then grab the image dimensions
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
all_images = os.listdir(image_folder)
true_labels = []
predictions = []
print("Total images in the specified folder: ", len(all_images))
start = time.time()
for i in range(len(all_images)):
    time_1 = time.time()
    file_name = all_images[i]
    true_name = file_name.split()[0]
    true_labels.append(true_name)
    image_path = os.path.join(image_folder, file_name)
    image = cv2.imread(image_path)

    image = imutils.resize(image, width=600)
    (h, w) = image.shape[:2]
    
    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(image, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)
    
    # apply OpenCV's deep learning-based face detector to localize faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()
    max_conf_detection_idx = np.argmax(detections[0, 0, :, 2])
    box = detections[0, 0, max_conf_detection_idx, 3:7] * np.array([w, h, w, h])
    (startX, startY, endX, endY) = box.astype("int")

    # Need to convert because the face_recognition.face_encoding() doesn't use coordinate system
    box = [(startY, endX, endY, startX)]
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Compute encoding using face_recognition library
    encodings = np.array(face_recognition.face_encodings(rgb, box))

    # perform classification to recognize the face
    preds = recognizer.predict_proba(encodings)[0]
    j = np.argmax(preds)
    proba = preds[j]
    name = le.classes_[j]
    predictions.append(name)

    time_2 = time.time()
    print(f"{time_2-time_1} sec on {file_name}, Predicted: {name}")
    
end = time.time()
print(len(true_labels), len(predictions))
print("[INFO] inference on {} images took {:.4f} seconds".format(len(all_images), end - start))


Total images in the specified folder:  49
0.452211856842041 sec on person_a (20).jpg, Predicted: person_a
0.4298722743988037 sec on person_a (21).jpg, Predicted: person_a
0.45113611221313477 sec on person_b (19).jpg, Predicted: person_b
0.425076961517334 sec on person_b (20).jpg, Predicted: person_b
0.4304018020629883 sec on person_c (21).jpg, Predicted: person_c
0.43249082565307617 sec on person_c (22).jpg, Predicted: person_c
0.42384767532348633 sec on person_d (20).jpg, Predicted: person_d
0.43385791778564453 sec on person_d (21).jpg, Predicted: person_d
0.4228851795196533 sec on person_e (22).jpg, Predicted: person_e
0.4996931552886963 sec on person_e (23).jpg, Predicted: person_e
0.4299647808074951 sec on person_f (19).jpg, Predicted: person_f
0.43134140968322754 sec on person_f (20).jpg, Predicted: person_f
0.4987504482269287 sec on person_g (10).jpg, Predicted: person_g
0.48669862747192383 sec on person_g (13).jpg, Predicted: person_g
0.4747314453125 sec on person_i (17).jpg, Pr

In [44]:
# show the classification report
print(classification_report(true_labels, predictions, target_names=le.classes_))

                      precision    recall  f1-score   support

            person_a       1.00      1.00      1.00         2
            person_b       1.00      1.00      1.00         2
            person_c       1.00      1.00      1.00         2
            person_d       1.00      1.00      1.00         2
            person_e       1.00      1.00      1.00         2
            person_f       1.00      1.00      1.00         2
            person_g       1.00      1.00      1.00         2
            person_i       1.00      1.00      1.00         2
            person_j       1.00      1.00      1.00         2
            person_k       1.00      1.00      1.00         3
            person_l       1.00      1.00      1.00         2
            person_m       1.00      1.00      1.00         2
            person_n       1.00      1.00      1.00         2
            person_o       1.00      1.00      1.00         5
            person_p       1.00      1.00      1.00         2
       